<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2022TeamA/blob/Konrad/COVID_Procedures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
# install the newest version 
!pip3 install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip


In [93]:
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas_profiling import ProfileReport

In [94]:
from google.colab import drive
# mount drive to access database
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [95]:
material_path = "/gdrive/Shareddrives/Gruppe A"

In [96]:
!ls "/content/gdrive/Shareddrives/Gruppe A/Datawarehouse.db"

'/content/gdrive/Shareddrives/Gruppe A/Datawarehouse.db'


In [97]:
# datawarehouse path
DB_DWH_PATH = f"/content/gdrive/Shareddrives/Gruppe A/Datawarehouse.db"
print("Datawarehouse: ", DB_DWH_PATH)
# connect to db
dwh_conn = sqlite3.connect(DB_DWH_PATH)

Datawarehouse:  /content/gdrive/Shareddrives/Gruppe A/Datawarehouse.db


In [98]:
# list of tables in db
if dwh_conn is not None:
  dwh_cursor = dwh_conn.cursor()
  dwh_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
  print("List of Tables", dwh_cursor.fetchall())

List of Tables [('Zentrum',), ('dimObservations',), ('dimEncounters',), ('dimProcedures',), ('dimConditions',), ('dimPatients',)]


In [99]:
# Read sqlite query results into a pandas DataFrame

# Zentrum Tabelle
df_Zentrum = pd.read_sql_query("SELECT * FROM Zentrum", dwh_conn)

# demographic data
df_patients = pd.read_sql_query("SELECT * FROM dimPatients", dwh_conn)
# diagnoses data
df_conditions = pd.read_sql_query("select * from dimConditions", dwh_conn)

# procedure data
# df_careplans = pd.read_sql_query("select * from careplans_info", dwh_conn)
df_procedures = pd.read_sql_query("SELECT * FROM dimProcedures", dwh_conn)

# observations
df_observations = pd.read_sql_query("SELECT * FROM dimObservations", dwh_conn)

# encounters
df_encounters = pd.read_sql_query("SELECT * FROM dimEncounters", dwh_conn)

Basic Data Exploration

In [100]:
df_procedures.head()

,DATE,PATIENT_ID,ENCOUNTER_ID,CODE,DESCRIPTION
0,2012-12-12T14:56:19Z,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,f222d3d6-d9f0-3586-70d4-69a77f16b58c,73761001,Colonoscopy
1,2015-05-04T11:40:19Z,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,c57b93a2-0302-4965-9e2c-713b84e7c24b,430193006,Medication Reconciliation (procedure)
2,2017-12-11T15:25:19Z,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,11306017-08e3-1375-fc44-1265a8114949,73761001,Colonoscopy
3,2019-01-30T11:40:19Z,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,cd0f83e5-1e70-75ec-1be9-b254e2589e49,23426006,Measurement of respiratory function (procedure)
4,2019-05-27T11:40:19Z,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,bdeb0bc4-50bc-5aaa-ab7f-e43535a01bbe,430193006,Medication Reconciliation (procedure)


In [101]:
df_procedures.columns

Index(['DATE', 'PATIENT_ID', 'ENCOUNTER_ID', 'CODE', 'DESCRIPTION'], dtype='object')

In [102]:
df_procedures.shape

(13570, 5)

In [103]:
# number of patients in procedure table
df_procedures.groupby(["PATIENT_ID"]).size()

PATIENT_ID
0033cec2-67d3-39ec-b0f4-29f91a2918d6      1
003c0d44-9dfc-ca91-6121-373a8e265bd4     13
00680fcd-0ef2-99c5-f69d-59463206750f     26
00edcb46-afd4-4687-c16a-f4e20dfc3e92      2
00f4a426-4746-6513-1775-1e8650e2f6b5      1
                                       ... 
ff2d586c-f770-a795-c9eb-0b0d39447e48      1
ff5c1f25-14da-ba3a-51bb-c5735d406ea8    179
ff9e749b-0abc-22f8-0d62-6912b87781b1      1
ffd905f3-45fb-b2a7-7429-bf864ef29d81     28
ffe52b9f-cdf1-d315-9286-e64d370edde4      1
Length: 1740, dtype: int64

In [104]:
df_procedures.groupby(["PATIENT_ID", "CODE"]).size()

PATIENT_ID                            CODE     
0033cec2-67d3-39ec-b0f4-29f91a2918d6  261352009    1
003c0d44-9dfc-ca91-6121-373a8e265bd4  180325003    7
                                      18286008     2
                                      40701008     1
                                      433112001    1
                                                  ..
ffd905f3-45fb-b2a7-7429-bf864ef29d81  261352009    1
                                      371908008    9
                                      399208008    1
                                      431182000    9
ffe52b9f-cdf1-d315-9286-e64d370edde4  261352009    1
Length: 4591, dtype: int64

Data Cleaning

In [105]:
# Nullwerte durch nan ersetzen
df_procedures = df_procedures.replace(r'^\s*$', np.nan, regex=True)

In [106]:
# Anzahl an Nullwerten im df_Procedures
df_procedures.isnull().sum()

DATE            0
PATIENT_ID      0
ENCOUNTER_ID    0
CODE            0
DESCRIPTION     0
dtype: int64

In [107]:
print("Number of Duplicated Rows", df_Zentrum.duplicated(df_Zentrum.columns).sum())

Number of Duplicated Rows 0


In [108]:
df_procedures.nunique(axis=0)

DATE            9296
PATIENT_ID      1740
ENCOUNTER_ID    7317
CODE             132
DESCRIPTION      137
dtype: int64

In [109]:
df_procedures.iloc[0]

DATE                            2012-12-12T14:56:19Z
PATIENT_ID      3575b903-dbd0-1d55-6146-9e8aa4ed52a5
ENCOUNTER_ID    f222d3d6-d9f0-3586-70d4-69a77f16b58c
CODE                                        73761001
DESCRIPTION                              Colonoscopy
Name: 0, dtype: object

In [110]:
entry=df_procedures.iloc[0]
print(type(entry))

<class 'pandas.core.series.Series'>


# Umwandlung des Datums in YYYY

In [111]:
# Abfrage des aktuellen Datumformats

from datetime import datetime
curr= datetime.now()
print(curr)

2022-01-30 10:39:14.711565


In [112]:
# Formatierung zu YYYY
req_format = datetime.strftime(curr, "%Y")
print(req_format)

2022


In [113]:
df_procedures["DATE"] = pd.to_datetime(df_procedures["DATE"])

In [114]:
df_procedures["DATE"] = df_procedures["DATE"].dt.to_period('Y')

/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/datetimes.py:1092: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


In [115]:
df_procedures.iloc[0]

DATE                                            2012
PATIENT_ID      3575b903-dbd0-1d55-6146-9e8aa4ed52a5
ENCOUNTER_ID    f222d3d6-d9f0-3586-70d4-69a77f16b58c
CODE                                        73761001
DESCRIPTION                              Colonoscopy
Name: 0, dtype: object

In [116]:
df_procedures.head

<bound method NDFrame.head of        DATE  ...                                        DESCRIPTION
0      2012  ...                                        Colonoscopy
1      2015  ...              Medication Reconciliation (procedure)
2      2017  ...                                        Colonoscopy
3      2019  ...    Measurement of respiratory function (procedure)
4      2019  ...              Medication Reconciliation (procedure)
...     ...  ...                                                ...
13565  1984  ...                 Percutaneous coronary intervention
13566  1984  ...                    Coronary artery bypass grafting
13567  2017  ...                       Echocardiography (procedure)
13568  2017  ...  Insertion of biventricular implantable cardiov...
13569  2017  ...               Catheter ablation of tissue of heart

[13570 rows x 5 columns]>

# Abfrage der Proceduren nach Jahren aufgeschlüsselt

In [117]:
# Erstellung mit einer Tabelle aus dem Corona-Jahr
covid_procedures = df_procedures[df_procedures["DATE"] >= "2020"]
covid_procedures.head()

,DATE,PATIENT_ID,ENCOUNTER_ID,CODE,DESCRIPTION
5,2020,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,3c7f472a-2602-bcdd-76d0-53f967923ded,261352009,Face mask (physical object)
15,2020,17f0c6d9-8931-8839-66cb-3ca6fb066d3e,dcee0f8b-907d-69cf-c998-5e73711f2276,261352009,Face mask (physical object)
16,2020,17f0c6d9-8931-8839-66cb-3ca6fb066d3e,61d084c0-52cc-1b9d-5ed8-505982d8f401,430193006,Medication Reconciliation (procedure)
26,2020,e92765c7-1c4b-9ee7-dbc8-5300fcb40a54,621750b5-9dd3-649c-977c-45d6315b546e,430193006,Medication Reconciliation (procedure)
54,2020,aff157cc-b6d3-412b-ccbe-bfd5fac1c2d5,61c250a0-1668-e4ce-aefc-77a7842574c2,261352009,Face mask (physical object)


In [118]:
# Erstellung einer Tabelle vor Corona
pre_covid_procedures = df_procedures[df_procedures["DATE"] < "2020"]
pre_covid_procedures.head()

,DATE,PATIENT_ID,ENCOUNTER_ID,CODE,DESCRIPTION
0,2012,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,f222d3d6-d9f0-3586-70d4-69a77f16b58c,73761001,Colonoscopy
1,2015,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,c57b93a2-0302-4965-9e2c-713b84e7c24b,430193006,Medication Reconciliation (procedure)
2,2017,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,11306017-08e3-1375-fc44-1265a8114949,73761001,Colonoscopy
3,2019,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,cd0f83e5-1e70-75ec-1be9-b254e2589e49,23426006,Measurement of respiratory function (procedure)
4,2019,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,bdeb0bc4-50bc-5aaa-ab7f-e43535a01bbe,430193006,Medication Reconciliation (procedure)


In [119]:
# Anzahl der im Corona-Jahr behandelten Patienten
covid_procedures.groupby(["PATIENT_ID"]).size()

PATIENT_ID
0033cec2-67d3-39ec-b0f4-29f91a2918d6     1
003c0d44-9dfc-ca91-6121-373a8e265bd4     1
00680fcd-0ef2-99c5-f69d-59463206750f    26
00edcb46-afd4-4687-c16a-f4e20dfc3e92     1
00f4a426-4746-6513-1775-1e8650e2f6b5     1
                                        ..
ff2d586c-f770-a795-c9eb-0b0d39447e48     1
ff5c1f25-14da-ba3a-51bb-c5735d406ea8     1
ff9e749b-0abc-22f8-0d62-6912b87781b1     1
ffd905f3-45fb-b2a7-7429-bf864ef29d81    21
ffe52b9f-cdf1-d315-9286-e64d370edde4     1
Length: 1219, dtype: int64

In [120]:
# Anzahl der prä-COVID behandelten Patienten
pre_covid_procedures.groupby(["PATIENT_ID"]).size()

PATIENT_ID
003c0d44-9dfc-ca91-6121-373a8e265bd4     12
00edcb46-afd4-4687-c16a-f4e20dfc3e92      1
01518fe2-bd76-d1e9-a656-1e7eae35dd17      2
01581211-2e47-027b-71a0-dcbf0f4e6622      2
01a49388-c6c0-1d10-00ee-8360a11941e7      3
                                       ... 
fda2e613-dd00-2f49-6e86-0c1e5c281658      2
fe55dd2f-bc68-7056-1b29-1d4c4563d30b      9
feb25ae2-c154-4c42-3276-bc6c1e337a6c      2
ff5c1f25-14da-ba3a-51bb-c5735d406ea8    178
ffd905f3-45fb-b2a7-7429-bf864ef29d81      7
Length: 826, dtype: int64

In [121]:
# Verteilung der durchgeführten Prozeduren im Corona-Jahr
covid_procedures.groupby(["DESCRIPTION"]).size()

DESCRIPTION
Alpha-fetoprotein test                                 1
Ankle X-ray                                            1
Asthma screening                                       1
Auscultation of the fetal heart                       10
Bilateral tubal ligation                               1
                                                      ..
Urine culture                                          2
Urine protein test                                     2
Urine screening for glucose                            1
Urine screening test for diabetes                      2
Vaccination for diphtheria  pertussis  and tetanus     1
Length: 84, dtype: int64

In [122]:
# Verteilung der durchgeführten Prozeduren vor COVID
pre_covid_procedures.groupby(["DESCRIPTION"]).size()

DESCRIPTION
Admission to burn unit                                 1
Admission to long stay hospital                        1
Admission to orthopedic department                     8
Allergy screening test                                 6
Alpha-fetoprotein test                                43
                                                      ..
Urine protein test                                    43
Urine screening for glucose                           43
Urine screening test for diabetes                     43
Vaccination for diphtheria  pertussis  and tetanus    43
X-ray or wrist                                         3
Length: 130, dtype: int64

In [138]:
# Count Procedures during COVID
covid_procedures.count()

DATE            4984
PATIENT_ID      4984
ENCOUNTER_ID    4984
CODE            4984
DESCRIPTION     4984
dtype: int64

In [139]:
# Count Procedures pre_COVID
pre_covid_procedures.count()

DATE            8586
PATIENT_ID      8586
ENCOUNTER_ID    8586
CODE            8586
DESCRIPTION     8586
dtype: int64

In [141]:
# Most often performed Procedures during COVID
covid_procedures['DESCRIPTION'].value_counts()[:20]

Oxygen administration by mask (procedure)                                              1544
Placing subject in prone position (procedure)                                          1504
Face mask (physical object)                                                             995
Plain chest X-ray (procedure)                                                           163
Controlled ventilation procedure and therapy  initiation and management (procedure)     152
Electrical cardioversion                                                                144
Hemodialysis (procedure)                                                                 63
Colonoscopy                                                                              55
Catheter ablation of tissue of heart                                                     44
Medication Reconciliation (procedure)                                                    42
Echocardiography (procedure)                                                    

In [162]:
# Most often performed procedures pre-COVID
pre_covid_procedures['DESCRIPTION'].value_counts()[:20]

Renal dialysis (procedure)                                                         1174
Electrical cardioversion                                                           1131
Medication Reconciliation (procedure)                                               702
Colonoscopy                                                                         490
Echocardiography (procedure)                                                        396
Auscultation of the fetal heart                                                     387
Evaluation of uterine fundal height                                                 387
Catheter ablation of tissue of heart                                                379
Intramuscular injection                                                             191
Combined chemotherapy and radiation therapy (procedure)                             160
Percutaneous mechanical thrombectomy of portal vein using fluoroscopic guidance     154
Subcutaneous immunotherapy      

# Graphische Darstellung

In [163]:
sns.countplot(x="DATE", data=df_procedures)
